In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from dataset import make_dataset
from os.path import join


train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')

x_train, y_train, test = make_dataset(train_path, test_path)

Start time:  2022-10-20 11:37:46.178534
Train dataset success !
Test dataset success !
End time: 2022-10-20 11:38:52.505503
Play time:  0:01:06.326969


In [2]:
X = x_train.copy()
y = y_train.copy()

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True, random_state=1103)

In [1]:
# import optuna
# from optuna import Trial
# from optuna.samplers import TPESampler


# def objective_xgb(trial: Trial, x, y):
#     params = {
#         "n_estimators": trial.suggest_int('n_estimators', 500, 5000),
#         'max_depth': trial.suggest_int('max_depth', 8, 16),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
#         'gamma': trial.suggest_int('gamma', 1, 3),
#         'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
#         'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
#         'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
#         'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 1.0]),
#         'random_state': 42
#     }

#     x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

#     model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0)
#     xgb_model = model.fit(x_train, y_train, verbose=False, eval_set=[(x_val, y_val)], early_stopping_rounds=50)
#     y_pred = xgb_model.predict(x_val)
#     score = mean_absolute_error(y_val, y_pred)

#     return score


# study = optuna.create_study(direction='minimize', sampler=TPESampler())
# study.optimize(lambda trial: objective_xgb(trial, X, y), n_trials=100)
# print('Best trial: score {},\nparams {}'.format(study.best_trial.value, study.best_trial.params))

# params = study.best_trial.params
# xgb_model = XGBRegressor(**params, tree_method='gpu_hist', gpu_id=0).fit(x_train, y_train)
# y_pred = xgb_model.predict(test)

# import pandas as pd

# sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
# sample_submission['target'] = y_pred
# sample_submission.to_csv("./submit_xgb_optuna.csv", index=False)

In [10]:
param = {'n_estimators': 3516, 'max_depth': 16, 'min_child_weight': 65, 'gamma': 2, 'learning_rate': 0.016, 'colsample_bytree': 0.6669214248431368, 'lambda': 2.4870659647843234, 'alpha': 0.37341889508206516, 'subsample': 0.7}

xgb_model = XGBRegressor(**param, tree_method='gpu_hist', gpu_id=0).fit(x_train, y_train)
y_pred = xgb_model.predict(test)

import pandas as pd

sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = y_pred
sample_submission.to_csv("./submit_xgbealy.csv", index=False)

In [12]:
df_imp = pd.DataFrame({'imp':xgb_model.feature_importances_}, index = xgb_model.feature_names_in_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()

In [18]:
b = pd.read_csv("public 3.17 cat_optuna.csv")

In [17]:
a = pd.read_csv('submit_xgbealy.csv')

In [19]:
c = (a['target']*0.5) + (b['target']*0.5)
sample_submission = pd.read_csv('./jeju_data/sample_submission.csv')
sample_submission['target'] = c
sample_submission.to_csv("./submit.csv", index=False)